In [1]:
library("cli")
library(stringr)
library("DESeq2")

parallel <- FALSE


dds <- readRDS("/mnt/disk08/users/nina/mouse_acetaminophen/30-940567013/Analises_PUCPR/results/deseq2/all.rds")



Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics




Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package

In [2]:
contrast_config <- list(
  "variable_of_interest" = "condition",
  "level_of_interest" = "exposed"
  )

# basic case of contrast specification, see:
# https://www.bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#contrasts
if (length(contrast_config) == 2 && typeof(contrast_config) == "list") {
  
  contrast <- c(
    contrast_config[["variable_of_interest"]],
    contrast_config[["level_of_interest"]],
    "control"
  )
# more complex contrast specification via list(c(), c()), see ?results docs of
# the DESeq2 package and this tutorial (plus the linked seqanswers thread):
# https://github.com/tavareshugo/tutorial_DESeq2_contrasts/blob/main/DESeq2_contrasts.md
} else if (
    length(contrast_config) == 1 &&
    typeof(contrast_config) == "character"
  ) {
  contrast <- d <- eval(parse(text = contrast_config))
}



In [3]:
coldata <- read.table("/mnt/disk08/users/nina/mouse_acetaminophen/30-940567013/Analises_PUCPR/config/samples.tsv", header=TRUE, check.names=FALSE)
coldata <- coldata[order(row.names(coldata)), , drop=F]

norm_counts = counts(dds, normalized=T)

res <- results(
  dds,
  contrast = contrast,
  parallel = parallel
)


# shrink fold changes for lowly expressed genes
# use ashr so we can use `contrast` as conversion to coef is not trivial, see:
# https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#extended-section-on-shrinkage-estimators
res <- lfcShrink(
  dds,
  contrast = contrast,
  res = res,
  type = "ashr"
)

# sort by p-value
res <- res[order(res$padj), ]


res <- data.frame(
    "gene" = rownames(res),
    res
  )

head(res)
conditions <- list("condition")


using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041



,gene,baseMean,log2FoldChange,lfcSE,pvalue,padj
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000078350,ENSMUSG00000078350,321.74496,0.10342277,0.07076200,8.933659e-06,0.2639819
ENSMUSG00000032498,ENSMUSG00000032498,120.95535,-0.09902010,0.06557843,1.715774e-05,0.2639819
ENSMUSG00000045257,ENSMUSG00000045257,39.71161,-0.06877913,0.11602582,2.498251e-05,0.2639819
ENSMUSG00000057751,ENSMUSG00000057751,91.38927,-0.08552145,0.06136863,8.550725e-05,0.2784303
ENSMUSG00000078490,ENSMUSG00000078490,153.92312,-0.05219260,0.10826884,7.080129e-05,0.2784303
ENSMUSG00000031492,ENSMUSG00000031492,209.59204,0.06223720,0.08010067,7.282112e-05,0.2784303


In [4]:
for (condition in conditions)
{
  for (i in unique(coldata[[condition]]))
  {
      mean <- data.frame(mean=rowMeans(norm_counts[,c(coldata[coldata[[condition]]==i,]$sample_name)]))
      mean <- cbind(rownames(mean), mean)
      rownames(mean) <- NULL
      colnames(mean) <- c("gene", paste("baseMean_", condition, "_", i, sep=""))
      head(mean)
      res<-merge(res, mean)
  }
}


# TODO explore IHW usage


#write.table(
#  res,
#  file = snakemake@output[["table"]],
#  row.names = FALSE,
#  sep = "\t"
#)

In [5]:
norm_counts

,CORTEX_A1,CORTEX_A2,CORTEX_A3,CORTEX_A4,CORTEX_A5,CORTEX_A6,CORTEX_C1,CORTEX_C2,CORTEX_C3,CORTEX_C4,CORTEX_C5,CORTEX_C6
ENSMUSG00000102135,0.000000,0.000000,0.000000,1.066221,0.9075376,0.000000,0.9287371,0.9515843,0.000000,0.863847,0.0000000,1.012780
ENSMUSG00000100764,3.418338,3.151722,5.023128,5.331105,2.7226127,4.151191,0.9287371,0.9515843,4.106789,0.863847,1.9539451,4.051120
ENSMUSG00000100635,3.418338,3.151722,3.013877,0.000000,2.7226127,0.000000,1.8574742,1.9031685,0.000000,1.727694,4.8848628,0.000000
ENSMUSG00000100480,12.533905,9.455166,12.055508,13.860873,9.0753757,13.491372,8.3586341,13.3221796,11.293670,6.910776,14.6545885,9.115021
ENSMUSG00000100025,21.649472,12.606887,18.083262,25.589303,17.2432139,33.209531,14.8597939,20.9348537,10.266972,23.323869,15.6315611,21.268382
ENSMUSG00000051285,3020.671088,2935.303617,2891.312601,2745.519007,3311.6045980,2444.013908,2988.6760478,2400.8470879,2197.132093,2393.720085,2983.6742277,3037.327523
ENSMUSG00000098201,0.000000,0.000000,0.000000,0.000000,0.0000000,1.037798,0.0000000,0.0000000,2.053394,2.591541,0.0000000,0.000000
ENSMUSG00000103509,14.812797,52.528698,4.018503,1.066221,14.5206011,2.075596,38.0782219,28.5475278,3.080092,0.863847,5.8618354,5.063901
ENSMUSG00000048538,7.976121,5.252870,5.023128,5.331105,6.3527630,13.491372,9.2873712,7.6126741,17.453853,6.910776,8.7927531,9.115021
ENSMUSG00000097797,10.255013,21.011479,16.074010,18.125757,12.7055260,12.453574,14.8597939,16.1769324,28.747523,25.051563,17.5855063,19.242822


In [6]:
res

gene,baseMean,log2FoldChange,lfcSE,pvalue,padj,baseMean_condition_control,baseMean_condition_exposed
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000000001,1025.8417720,0.0003766700,0.05208385,0.9898677,0.9970818,1025.7103830,1025.973161
ENSMUSG00000000028,34.6246822,-0.0043589787,0.06181734,0.7480736,0.9924186,35.7304321,33.518932
ENSMUSG00000000031,5.0886704,0.0008933902,0.06311290,0.8773040,0.9924186,4.9020689,5.275272
ENSMUSG00000000037,9.4294867,-0.0032321132,0.06308866,0.5950781,0.9924186,10.5402907,8.318683
ENSMUSG00000000049,22.4068786,0.0021746376,0.06273615,0.8054852,0.9924186,21.5661225,23.247635
ENSMUSG00000000056,1394.3441132,-0.0157639985,0.05841230,0.4861199,0.9924186,1445.0488363,1343.639390
ENSMUSG00000000058,948.5350012,0.0197767410,0.05914002,0.3515197,0.9924186,898.2087266,998.861276
ENSMUSG00000000078,1675.9958078,0.0080383953,0.05728173,0.7426863,0.9924186,1650.5012866,1701.490329
ENSMUSG00000000085,930.6526054,0.0267551168,0.05657667,0.2935627,0.9924186,888.0705341,973.234677


In [7]:
res<-merge(res, norm_counts,by.x="gene", by.y='row.names')

In [8]:
res

gene,baseMean,log2FoldChange,lfcSE,pvalue,padj,baseMean_condition_control,baseMean_condition_exposed,CORTEX_A1,CORTEX_A2,CORTEX_A3,CORTEX_A4,CORTEX_A5,CORTEX_A6,CORTEX_C1,CORTEX_C2,CORTEX_C3,CORTEX_C4,CORTEX_C5,CORTEX_C6
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000000001,1025.8417720,0.0003766700,0.05208385,0.9898677,0.9970818,1025.7103830,1025.973161,925.230073,1106.254370,1076.958689,969.194865,1080.8772475,997.323722,1002.1073510,1046.742686,1010.270084,871.621640,1064.900101,1158.620436
ENSMUSG00000000028,34.6246822,-0.0043589787,0.06181734,0.7480736,0.9924186,35.7304321,33.518932,19.370580,28.365497,37.171149,28.787966,27.2261271,60.192275,25.0759022,39.966539,45.174679,40.600810,34.194040,29.370623
ENSMUSG00000000031,5.0886704,0.0008933902,0.06311290,0.8773040,0.9924186,4.9020689,5.275272,7.976121,3.151722,6.027754,8.529768,1.8150751,4.151191,2.7862114,0.000000,7.186881,4.319235,1.953945,13.166141
ENSMUSG00000000037,9.4294867,-0.0032321132,0.06308866,0.5950781,0.9924186,10.5402907,8.318683,14.812797,1.050574,10.046256,14.927094,9.0753757,0.000000,13.9310568,21.886438,4.106789,10.366164,5.861835,7.089461
ENSMUSG00000000049,22.4068786,0.0021746376,0.06273615,0.8054852,0.9924186,21.5661225,23.247635,22.788918,24.163201,20.092513,7.463547,24.5035144,40.474116,16.7172681,3.806337,36.961101,30.234646,23.447342,18.230042
ENSMUSG00000000056,1394.3441132,-0.0157639985,0.05841230,0.4861199,0.9924186,1445.0488363,1343.639390,1345.685611,1239.677261,1570.229880,1309.319355,1378.5495709,1218.374662,1401.4643120,1946.941396,968.175497,1364.878287,1488.906196,1499.927330
ENSMUSG00000000058,948.5350012,0.0197767410,0.05914002,0.3515197,0.9924186,898.2087266,998.861276,906.998939,984.387792,1056.866177,1044.896554,1010.9968545,989.021339,851.6519378,511.952332,1147.847514,920.860920,960.364036,996.575619
ENSMUSG00000000078,1675.9958078,0.0080383953,0.05728173,0.7426863,0.9924186,1650.5012866,1701.490329,1592.945372,1715.587261,1809.330783,1863.754262,1483.8239292,1743.500368,1449.7586422,1223.737358,1821.360903,1643.037027,1641.313917,2123.799872
ENSMUSG00000000085,930.6526054,0.0267551168,0.05657667,0.2935627,0.9924186,888.0705341,973.234677,1022.082975,1046.371655,1050.838423,943.605562,999.1988661,777.310581,1081.0500061,920.181979,701.234215,800.786185,963.294954,861.875866
